In this weeks assignments you will join and aggregate data from a collection of user and restaurant in Mexico. The collections we use is from: https://archive.ics.uci.edu/ml/datasets/Restaurant+%26+consumer+data

We have placed three files in the folder of this tutorial: 
- `userprofile.csv`: describes several atributes of the users
- `geoplaces2.csv`:  describes several attributes of the restaurants
- `rating_final.csv`: describes how the users rated the restaurant, the food and the service on a scale [0-2]

## part 1 ##

The first assignment is to estimate whether people with a larger budget are generally more satisfied with the restaurant they choose.

Create an RDD `rating` of the restaurant rating (the first rating) in `rating_final.csv`.

In [18]:
ratings_final = sc.textFile("rating_final.csv")
firstline = ratings_final.first()
ratings_finalNoHeader = ratings_final.filter(lambda x: x != firstline)
ratingsSplit = ratings_finalNoHeader.map(lambda x: x.split(','))
rating = ratingsSplit.map(lambda x: (x[0], x[1], x[2] ) ) 
rating.take(5)

[('U1077', '135085', '2'),
 ('U1077', '135038', '2'),
 ('U1077', '132825', '2'),
 ('U1077', '135060', '1'),
 ('U1068', '135104', '1')]

Also create an RDD `userbudget` in which you load the `userID` and `budget` from `userprofiles.csv`.

In [19]:
user_profile_final = sc.textFile("userprofile.csv")
firstline = user_profile_final.first()
user_profile_final_NoHeader = user_profile_final.filter(lambda x: x != firstline)
user_profile_Split = user_profile_final_NoHeader.map(lambda x: x.split(','))
user_profile_Split.take(5)
userbudget = user_profile_Split.map(lambda x: (x[0], x[17]) ) 
userbudget.take(5)

[('U1001', 'medium'),
 ('U1002', 'low'),
 ('U1003', 'low'),
 ('U1004', 'medium'),
 ('U1005', 'medium')]

To join the `userbudget` with the `userrating`, you must convert the `ratings` to a (userID, rating) structure (we don't need the placeID for this assignment).

In [21]:
firstlineFilter = rating.first()
ratings_final_Final = rating.filter(lambda x: x != firstlineFilter)
useratings = ratings_final_Final.map(lambda x: (x[0], x[2]))
useratings.take(5)

[('U1077', '2'),
 ('U1077', '2'),
 ('U1077', '2'),
 ('U1077', '1'),
 ('U1068', '1')]

Join `useratings` and `userbudget`, and map the result to a `(budget, rating)` structure. Don't forget to convert rating to an int (with `int()`).

In [42]:
users_union = userbudget.leftOuterJoin(useratings) 
union_convert = users_union.map(lambda x:  (x[1][0], int(x[1][1]) ) )
union_convert.take(5)


[('low', 1), ('low', 1), ('low', 2), ('low', 1), ('low', 1)]

Group the result by budget (the key), and compute the average rating. To compute the average of a list `l` in python you can divide `sum(l)` by `len(l)`.

In [60]:
budget_group = union_convert.aggregateByKey(\
                  0, # initial value for an accumulator \
                  lambda r, v: (r + v) / 2 , # function that adds a value to an accumulator \
                  lambda r1, r2: (r1 + r2) / 2 # function that merges/combines two accumulators \
                 )
budget_group.collect()

budget_group_other = union_convert.aggregateByKey(\
                  0, # initial value for an accumulator \
                  lambda r, v: (r + v) / 1 , # function that adds a value to an accumulator \
                  lambda r1, r2: (r1 + r2) / 1 # function that merges/combines two accumulators \
                 )

budget_group_other.collect()

final = budget_group_other.map(lambda x:  (x[0], x[1] / (union_convert.filter(lambda y: y[0] == x[0] )).count()      ) )
final.collect()

Traceback (most recent call last):
  File "/opt/jupyterhub/spark-2.2.0/python/pyspark/cloudpickle.py", line 148, in dump
    return Pickler.dump(self, obj)
  File "/opt/jupyterhub/anaconda/lib/python3.6/pickle.py", line 409, in dump
    self.save(obj)
  File "/opt/jupyterhub/anaconda/lib/python3.6/pickle.py", line 476, in save
    f(self, obj) # Call unbound method with explicit self
  File "/opt/jupyterhub/anaconda/lib/python3.6/pickle.py", line 751, in save_tuple
    save(element)
  File "/opt/jupyterhub/anaconda/lib/python3.6/pickle.py", line 476, in save
    f(self, obj) # Call unbound method with explicit self
  File "/opt/jupyterhub/spark-2.2.0/python/pyspark/cloudpickle.py", line 255, in save_function
    self.save_function_tuple(obj)
  File "/opt/jupyterhub/spark-2.2.0/python/pyspark/cloudpickle.py", line 292, in save_function_tuple
    save((code, closure, base_globals))
  File "/opt/jupyterhub/anaconda/lib/python3.6/pickle.py", line 476, in save
    f(self, obj) # Call unboun

PicklingError: Could not serialize object: Exception: It appears that you are attempting to broadcast an RDD or reference an RDD from an action or transformation. RDD transformations and actions can only be invoked by the driver, not inside of other transformations; for example, rdd1.map(lambda x: rdd2.values.count() * x) is invalid because the values transformation and count action cannot be performed inside of the rdd1.map transformation. For more information, see SPARK-5063.

Indeed, it seems that users with a higher budget are more satisfied with the restaurants they visit.

## Part 2 ##

The next assignment is to estimate whether users ratings are affected by the distance between where they live and where the restaurant is. 

We want to compute the distance between the user's home and the restaurant for every rating. Both positions can be looked up from the userprofiles and the places.

Create an RDD `userpos` with the userID, latitude and longitude from `userprofiles.csv`. To use the location, put latitude and longitude inside a tuple.

In [52]:
user_profile_final = sc.textFile("userprofile.csv")
user_profile_final.take(5)

['userID,latitude,longitude,smoker,drink_level,dress_preference,ambience,transport,marital_status,hijos,birth_year,interest,personality,religion,activity,color,weight,budget,height',
 'U1001,22.139997,-100.978803,false,abstemious,informal,family,on foot,single,independent,1989,variety,thrifty-protector,none,student,black,69,medium,1.77',
 'U1002,22.150087,-100.983325,false,abstemious,informal,family,public,single,independent,1990,technology,hunter-ostentatious,Catholic,student,red,40,low,1.87',
 'U1003,22.119847,-100.946527,false,social drinker,formal,family,public,single,independent,1989,none,hard-worker,Catholic,student,blue,60,low,1.69',
 'U1004,18.867,-99.183,false,abstemious,informal,family,public,single,independent,1940,variety,hard-worker,none,professional,green,44,medium,1.53']

Create a broadcast variable that contains a dictionary from which you can lookup a users position based on their ID.

Also create an RDD `placepos` that contains the ID and position of places in `geoplaces2.csv`.

And create a similar broadcast variable to lookup the position of a place based in it's ID.

To compute the distance an accurate approximation is the Vincenty distance in the geopy library (use `pip install geopy` to install). 

Here is an example to compute the distance:

In [ ]:
from geopy.distance import vincenty
vincenty((31.8300167,35.0662833), (31.83,35.0708167)).meters

Now, map the ratings, so that you retrieve the position of the user and the position of the restaurant, and compute the vincenti distance between them. Output the distance and rating.

Now average the distance per rating.

It seems that there is no linear relation between the distance to the restaurant and the given rating.